In [181]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier, callback

In [182]:
df = pd.read_csv('../../data/expanded_train_with_leaf.csv')
df = df[df.language == 'Finnish'].reset_index(drop=True)

In [183]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(df, df.is_root, groups=df.sentence))
train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

In [184]:
CENT = [
    'degree','closeness','harmonic','betweeness','load','pagerank',
    'eigenvector','katz','information','current_flow_betweeness',
    'percolation','second_order','laplacian','is_leaf'
]

In [185]:
X_train = train_df[CENT];  y_train = train_df['is_root'];  s_train = train_df['sentence']
X_val   = val_df[CENT];    y_val   = val_df['is_root'];  s_val   = val_df['sentence']

In [186]:
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)

In [187]:
def root_score(sent_ids, y_true, probs):
    tmp = pd.DataFrame({'sentence': sent_ids, 'is_root': y_true, 'p': probs})
    pick = tmp.loc[tmp.groupby('sentence')['p'].idxmax()]
    return (pick.is_root == 1).mean()

In [188]:
rf_grid = {
    'n_estimators':    [100, 300, 500],
    'max_depth':       [5, 10, 20, None],
    'min_samples_leaf':[5, 10, 20],
    'max_features':    ['sqrt', 'log2', 0.5],
}

In [189]:
best_rf_score = -1
best_rf = None
for ne in rf_grid['n_estimators']:
    for md in rf_grid['max_depth']:
        for msl in rf_grid['min_samples_leaf']:
            for mf in rf_grid['max_features']:
                rf = RandomForestClassifier(
                    n_estimators=ne,
                    max_depth=md,
                    min_samples_leaf=msl,
                    max_features=mf,
                    class_weight='balanced',
                    random_state=42,
                    n_jobs=-1
                )
                rf.fit(X_train_s, y_train)
                p = rf.predict_proba(X_val_s)[:,1]
                score = root_score(s_val, y_val, p)
                if score > best_rf_score:
                    best_rf_score = score
                    best_rf = rf

print(f"Best RF hold-out root-score: {best_rf_score:.3f}")

Best RF hold-out root-score: 0.370


In [190]:
lgb_grid = {
    'learning_rate':     [0.01, 0.05, 0.1],
    'num_leaves':        [16, 31, 63],
    'min_child_samples': [10, 20, 40],
}

best_lgb_score = -1
best_lgb = None
for lr in lgb_grid['learning_rate']:
    for nl in lgb_grid['num_leaves']:
        for mcs in lgb_grid['min_child_samples']:
            lgb = LGBMClassifier(
                objective='binary',
                n_estimators=500,
                learning_rate=lr,
                num_leaves=nl,
                min_child_samples=mcs,
                feature_fraction=0.8,
                bagging_fraction=0.8,
                bagging_freq=5,
                class_weight='balanced',
                random_state=42,
                n_jobs=-1
            )
            lgb.fit(
                X_train_s, y_train,
                eval_set=[(X_val_s, y_val)],
                eval_metric='binary_logloss',
                callbacks=[
                    callback.early_stopping(stopping_rounds=20),
                    callback.log_evaluation(period=0)
                ]
            )
            p = lgb.predict_proba(X_val_s)[:,1]
            score = root_score(s_val, y_val, p)
            if score > best_lgb_score:
                best_lgb_score = score
                best_lgb = lgb

print(f"Best LGB hold-out root-score: {best_lgb_score:.3f}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Number of positive: 400, number of negative: 5031
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000693 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3059
[LightGBM] [Info] Number of data point

D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 

...

D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[181]	valid_0's binary_logloss: 0.304282
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: baggin

D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[98]	valid_0's binary_logloss: 0.308667
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging

D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [191]:
p_rf  = best_rf.predict_proba(X_val_s)[:,1]
p_lgb = best_lgb.predict_proba(X_val_s)[:,1]
p_ens = 0.5 * p_rf + 0.5 * p_lgb
ens_score = root_score(s_val, y_val, p_ens)

print(f"Ensemble root-score: {ens_score:.3f}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Ensemble root-score: 0.350


D:\BDMA\Machine_Learning\ml-project-root-node-in-free-tree\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
